# Data cleaning

## Outline
- [Necessary packages](#necessary_packages)
- [Data loading](#data_loading)
- [Getting familiar with the data](#getting_familiar_with_the_data)
- [Columns renaming](#columns_renaming)
- [Handeling missing data](#handeling_missing_data)
- [Handeling duplicates](#handeling_duplicates)
- [Dropping irrelevant columns](#dropping_irrelevant_columns)
- [Types correction](#types_correction)
- [Text cleaning and preprocessing](#text_cleaning_and_preprocessing)
- [Labels preprocessing](#labels_preprocessing)
- [Save the results to the disk](#save_to_disk)
- [Notes](#notes)

<div id="necessary_packages" >
    <h3>Necessary packages</h3>
</div>

In [15]:
import numpy as np
import pandas as pd
import scipy as sp
import nltk
import os
import string
import re
import json
import shutil
from tqdm.notebook import tqdm

In [2]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/abdelnour/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/abdelnour/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abdelnour/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/abdelnour/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
tqdm.pandas()

<div id="data_loading" >
    <h3>Data loading</h3>
</div>

In [23]:
path = os.path.join("..","data", "stores_data.csv")

In [24]:
df = pd.read_csv(filepath_or_buffer=path,lineterminator="\n")

<div id="getting_familiar_with_the_data" >
    <h3>Getting familiar with the data</h3>
</div> 

In [6]:
df.sample(5)

,Unnamed: 0,store_title,store_description,store_collections,store_labels
150819,150819,Gossip Girl Jewelry,Gossip Girl Jewelry,"zi collection, frontpage, rings, necklaces, mi...",['jewelry']
439323,439323,"Uncommon Now - The shop for unique gadgets, gi...",Uncommon Now is stocked full of unique must-ha...,all products,"['gadget', 'gift']"
38269,38269,Nini's Jewels - Shop Fine Jewelry,Shop Jewelry - We just updated our new Alexand...,"cubic zirconia, rings, alexandrite",['jewelry']
430026,430026,Paper Thermometer™ | Temperature Indicating De...,"Improving product safety, accuracy, and qualit...","cabinets furniture, 3 temp thermolabel, 8 temp...",['food']
570476,570476,SafetyScan Technologies Fitness for Duty screener,"Impairment screening for alcohol, drugs and fa...",NaN,[]


In [7]:
df.columns

Index(['Unnamed: 0', 'store_title', 'store_description', 'store_collections',
       'store_labels'],
      dtype='object')

In [8]:
df.dtypes

Unnamed: 0            int64
store_title          object
store_description    object
store_collections    object
store_labels         object
dtype: object

In [9]:
df.shape

(618768, 5)

<div id="columns_renaming" >
    <h3>Columns renaming</h3>
</div>

In [35]:
df.columns = ["id","title","description","collections","labels"]

<div id="handeling_missing_data" >
    <h3>Handeling missing data</h3>
</div>

#### the number of missing values per row

In [11]:
na_df = df.isna()

In [12]:
na_df.sum()

id                 0
title            640
description    25831
collections    78804
labels             0
dtype: int64

#### the number of rows with at least one missing value

In [13]:
na_df[["title","description","collections"]].sum(axis=1).sum()

105275

#### the number of rows where all values are nan

In [14]:
na_df[["title","description","collections"]].prod(axis=1).sum()

0

#### the number of rows with both description and the list collections being nan

In [15]:
na_df[["description","collections"]].prod(axis=1).sum()

403

#### Imputing missing values

- Missing store description & store collection signifies that the user simply choose to not enter the description/list of collections for their store,hence they should be imputed with an empty string.
- Dropping the rows with both the description and the list of collections being nan should be eliminated since a great predection can not be derived from short text such as the title alone.
- Rows with an empty title can be dropped since the portion of such rows is very small.

In [33]:
def handle_missing_values(df: pd.DataFrame):
    na_df = df.isna()
    st_idx = df.index[na_df["title"]]
    sd_sc_idx = df.index[na_df[["description","collections"]].prod(axis=1) == 1]
    idx = pd.Index.union(st_idx,sd_sc_idx)
    df.drop(index=idx,inplace=True)
    df.fillna(value="",inplace=True)

In [36]:
handle_missing_values(df)

In [18]:
df.isna().sum()

id             0
title          0
description    0
collections    0
labels         0
dtype: int64

<div id="handeling_duplicates" >
    <h3>Handeling duplicates</h3>
</div>

In [19]:
df.duplicated().sum()

0

In [20]:
df["id"].duplicated().sum()

0

<div id="dropping_irrelevant_columns" >
    <h3>Dropping irrelevant columns</h3>
</div>

In [21]:
df.drop(columns=["id"],inplace=True)

<div id="types_correction" >
    <h3>Types correction</h3>
</div>

- Here the column "lables" is interpreted as string,it should be transformed to a list for more flexible manipulation.

In [22]:
def str_2_list(s):
    s = s[1:-1]
    if s != "":
        s = s.split(",")
        s = list(map(lambda x:x.strip()[1:-1], s))
        return s
    else:
        return []

In [23]:
df["labels"] = df["labels"].progress_apply(str_2_list)

  0%|          | 0/617725 [00:00<?, ?it/s]

In [24]:
df["labels"].head()

0                         []
1    [case, iphone, leather]
2       [bracelet, boutique]
3                  [fitness]
4                         []
Name: labels, dtype: object

<div id="text_cleaning_and_preprocessing" >
    <h3>Text Cleaning and preprocessing</h3>
</div>

- Capilization : Transforming the text to lower case.
- Noise Removal : Remove digits and spetial characters.
- Tokenization : the process of breaking a piece of text into a sequence of words or phrases.
- Stop words Removal : removing the most frequently used words that have no impact on the context of the text such as pronouncs.
- Stemming : the process replacing the words in a text with their root,for example : playing,player -> play.

In [25]:
stop_words = set(nltk.corpus.stopwords.words("english"))

In [26]:
spetial_chars = set(string.printable) - set(string.ascii_letters) - set(" ")
escaped_chars = [re.escape(c) for c in spetial_chars]
regex = re.compile(f"({'|'.join(escaped_chars)})")

In [27]:
stemmer = nltk.stem.porter.PorterStemmer()

In [28]:
def transform(text):

    # capitalization
    text = text.lower()
    
    # noise removal
    text = re.sub(regex," ",text)
    
    # tokenization
    text = nltk.word_tokenize(text, language='english')
        
    # stop words removal
    text = [word for word in text if word not in stop_words]
    
    # stemming
    text = [stemmer.stem(word) for word in text]
    
    return ' '.join(text)

In [29]:
df["title"] = df["title"].progress_apply(transform)

  0%|          | 0/617725 [00:00<?, ?it/s]

In [30]:
df["description"] = df["description"].progress_apply(transform)

  0%|          | 0/617725 [00:00<?, ?it/s]

In [31]:
df["collections"] = df["collections"].progress_apply(transform)

  0%|          | 0/617725 [00:00<?, ?it/s]

<div id="labels_preprocessing" >
    <h3>Labels preprocessing</h3>
</div>

#### Get all the possible labels

In [32]:
class Labels:

    def __init__(self):
        self.values = set()

    def add(self, x):
        self.values = self.values.union(set(x))

In [33]:
labels = Labels()

In [34]:
_ = df["labels"].progress_apply(labels.add)

  0%|          | 0/617725 [00:00<?, ?it/s]

In [35]:
print(f"the number of distinct labels is : {len(labels.values)}")

the number of distinct labels is : 724


#### Store the labels in a sprase matrix for memory efficiency

In [36]:
columns = []

In [37]:
for label in tqdm(labels.values):
    col = df["labels"].apply(lambda x: label in x).astype(np.int8)
    col = sp.sparse.csr_matrix(col.values)
    columns.append(col)
    del col

  0%|          | 0/724 [00:00<?, ?it/s]

In [38]:
exploded_labels = sp.sparse.vstack(columns)

#### Convert the sparse matrix to a sparse data frame

In [65]:
labels_df = pd.DataFrame.sparse.from_spmatrix(data=exploded_labels.T,index=df.index,columns=labels.values)

<div id="save_to_disk" >
    <h3>Save the results to the disk</h3>
</div>

In [62]:
sp.sparse.save_npz(os.path.join("data", "y.npz"), exploded_labels)

In [ ]:
labels_df.to_csv(os.path.join("data", "y.csv"))

In [63]:
with open(os.path.join("data","labels.txt"), mode="w") as f:
    f.write(','.join(labels.values))

In [58]:
df["labels"] = df["labels"].progress_apply(json.dumps)

  0%|          | 0/617725 [00:00<?, ?it/s]

In [59]:
df.to_csv(os.path.join("data", "data.csv"))

In [16]:
shutil.make_archive("data", 'zip', "data")

'/home/abdelnour/Documents/4eme_anne/S1/ML/mini-project/project/data-cleaning/data.zip'

<div id="notes" >
    <h3>Notes</h3>
</div>

- the zip file data contains 7 files:
    - `data.csv`: the preprocessed data set,of size `193.9MB`.
    - `y.csv`: the exploded labels in csv format, of size `851.1MB`.
    - `y.npz`: the exploded labels stored as a sparse scipy matrix,of size: `2.7MB`
    - `labels.txt`: the distinct labels stored as a comma seperated string,can be used in conjunction with `y.npz` instead of `y.csv` for memory efficiency.
    - `reduces_y.npz`: the exploded reduced labels as a sparse matrix, of size `964.6KB` (see `labels.ipynb` for more details).
    - `model.joblib`: the model used to map old labels to new labels, of size `1.7MB` (see `labels.ipynb` for more details).
    - `model2.joblib`: the model used to map old labels to new labels, of size `1.1MB` (see `labels.ipynb` for more details)
- The size of `data.zip` is : `94.1MB` (uncompressed : `1.1GB`),can be found at: https://drive.google.com/file/d/1HvovswN2RsHa72rb5TGhZEhZdTh2j1Pw/view?usp=sharing.
- How to use `y.npz` instead of `y.csv` for memory efficiency,however you'll most likely be able to achieve what you want throught the column `labels` which is so much faster:

  
```python
    import pandas as pd
    import scipy

    # read data.csv
    df = pd.read_csv("<path-to-data.csv>")

    # read the labels
    labels = None
    with open("<path-to-labels.txt>", "r") as f:
        labels = f.read().split(",")
        
    sparse_y = scipy.sparse.load_npz("<path-to-y.npz>")

    # convert to a sparse df
    df_y = pd.DataFrame.sparse.from_spmatrix(data=sparse_y.T,index=df.index,columns=labels)

    # get all the rows where "clothing is true":
    df[df_y["clothing"] == 1]
```
- How load/save the dataset:
    - since pandas doesn't serialize lists and confuse empty strings for nan values,use the functions `load_dataset` and `save_dataset` located in the file `utils.py` to load/save the dataset.

In [13]:
import pandas as pd
df = pd.read_csv("data/data.csv")

In [14]:
df.isna().sum()

title              216
description      25433
collections      78550
labels               0
mapped labels        0
dtype: int64

In [ ]:
df